<a href="https://colab.research.google.com/github/AkersJack/TextToLatex/blob/main/Text2Latex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mamba-ssm
!pip install datasets
!pip install trl
!pip install peft
!pip install numba

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, MambaForCausalLM, MambaConfig

from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel



In [ ]:
device = "cuda"
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/NLP_Project/mamba-chat-math2/checkpoint-8000")
#tokenizer = AutoTokenizer.from_pretrained("havenhq/mamba-chat")

tokenizer.eos_token = "<|endoftext|>"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.chat_template = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-beta").chat_template

model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/NLP_Project/mamba-chat-math2/checkpoint-8000", residual_in_fp32 = False)
#model.to("cuda")
#model = MambaLMHeadModel.from_pretrained("havenhq/mamba-chat", device="cuda", dtype=torch.float16)
model.to("cuda")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The fast path is not available because on of `(selective_state_update, selective_scan_fn, causal_conv1d_fn, causal_conv1d_update, mamba_inner_fn)` is None. Falling back to the naive implementation. To install follow https://github.com/state-spaces/mamba/#installation and https://github.com/Dao-AILab/causal-conv1d


MambaForCausalLM(
  (backbone): MambaModel(
    (embeddings): Embedding(50280, 1536)
    (layers): ModuleList(
      (0-47): 48 x MambaBlock(
        (norm): MambaRMSNorm()
        (mixer): MambaMixer(
          (conv1d): Conv1d(3072, 3072, kernel_size=(4,), stride=(1,), padding=(3,), groups=3072)
          (act): SiLU()
          (in_proj): Linear(in_features=1536, out_features=6144, bias=False)
          (x_proj): Linear(in_features=3072, out_features=128, bias=False)
          (dt_proj): Linear(in_features=96, out_features=3072, bias=True)
          (out_proj): Linear(in_features=3072, out_features=1536, bias=False)
        )
      )
    )
    (norm_f): MambaRMSNorm()
  )
  (lm_head): Linear(in_features=1536, out_features=50280, bias=False)
)

Chat with Mamba

In [ ]:
messages = []
while True:
    user_message = input("\nYour message: ")
    messages.append(dict(
        role="user",
        content=user_message
    ))

    input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to("cuda")
    #input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True)

    #out = model.generate(input_ids=input_ids, max_length=2000, temperature=0.9, top_p=0.7, eos_token_id=tokenizer.eos_token_id)
    out = model.generate(input_ids=input_ids, max_length=8000, temperature=0.9, top_p=0.7, eos_token_id=tokenizer.eos_token_id)

    decoded = tokenizer.batch_decode(out)
    messages.append(dict(
        role="assistant",
        content=decoded[0].split("<|assistant|>\n")[-1])
    )
    output = decoded[0].split("<|assistant|>\n")[-1]
    print("Model:", output.split("<|endoftext|>")[0])